# Notebook 2 – Clifford Algebra 101

**Level:** Beginner  
**Topics:** Wedge product, null property, anticommutativity, associativity, grade projections, multivectors

> **Provenance:** This notebook is based on:
> - `examples/wedge_product_example.py`
> - `examples/grade_projection_example.py`

## 1. Setup & Intuition

The **wedge product** `v ∧ w` builds oriented areas, volumes, and higher-dimensional analogues.

- Grade 0 – **Scalar** (volume of a 0D object)
- Grade 1 – **Vector** (direction in space)
- Grade 2 – **Bivector** (oriented area element)
- Grade 3 – **Trivector** (oriented 3D volume)
- Grade 4 – **Pseudoscalar** (oriented 4D volume in this model)

In [1]:
import jax.numpy as jnp
from light_theory_realm import CliffordEngine

engine = CliffordEngine(seed=42)
gammas = engine.gammas  # basis vectors

W1202 13:58:44.874207 1872096 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1202 13:58:44.881563 1871972 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.


## 2. Wedge Product: Null, Anticommutative, Grade Preserving

In [2]:
print("Null property: v ∧ v = 0")

for i in range(4):
    v = gammas[i]
    result = engine.wedge_product(v, v)
    norm = float(jnp.linalg.norm(result))
    print(f"‖g{i} ∧ g{i}‖ = {norm:.2e}")

Null property: v ∧ v = 0
‖g0 ∧ g0‖ = 0.00e+00
‖g1 ∧ g1‖ = 0.00e+00
‖g2 ∧ g2‖ = 0.00e+00
‖g3 ∧ g3‖ = 0.00e+00


In [3]:
print("Anticommutativity: v1 ∧ v2 = - v2 ∧ v1")

v1 = gammas[0]
v2 = gammas[1]

w12 = engine.wedge_product(v1, v2)
w21 = engine.wedge_product(v2, v1)

error = float(jnp.linalg.norm(w12 + w21))
print(f"‖g0 ∧ g1 + g1 ∧ g0‖ = {error:.2e}  (≈ 0 means perfect anticommutativity)")

Anticommutativity: v1 ∧ v2 = - v2 ∧ v1
‖g0 ∧ g1 + g1 ∧ g0‖ = 0.00e+00  (≈ 0 means perfect anticommutativity)


In [4]:
print("Grade preservation: grade(v ∧ B) = grade(v) + grade(B)")

v = gammas[0]  # grade 1
b = engine.wedge_product(gammas[1], gammas[2])  # grade 2
vb = engine.wedge_product(v, b)  # expected grade 3

for grade in range(5):
    component = engine.project_grade(vb, grade)
    norm = float(jnp.linalg.norm(component))
    expected = "nonzero" if grade == 3 else "≈ 0"
    print(f"  grade {grade}: ‖component‖ = {norm:.2e} ({expected})")

Grade preservation: grade(v ∧ B) = grade(v) + grade(B)
  grade 0: ‖component‖ = 0.00e+00 (≈ 0)
  grade 1: ‖component‖ = 0.00e+00 (≈ 0)
  grade 2: ‖component‖ = 0.00e+00 (≈ 0)
  grade 3: ‖component‖ = 2.00e+00 (nonzero)
  grade 4: ‖component‖ = 0.00e+00 (≈ 0)


## 3. Grades 0–4: What They Mean

Here's a qualitative summary:

- **Grade 0 (Scalar)** – A pure number. In Clifford form, often represented as a multiple of the identity matrix.
- **Grade 1 (Vector)** – A direction in spacetime (here: one of the γᵢ basis vectors or a combination).
- **Grade 2 (Bivector)** – An oriented area element (e.g. a "plane" spanned by two vectors).
- **Grade 3 (Trivector)** – An oriented 3D volume element.
- **Grade 4 (Pseudoscalar)** – An oriented 4D hyper-volume element (maximal grade in 4D).

We'll now *see* these grades numerically via projection.

## 4. Grade Projection Demo

In [5]:
# Build a mixed multivector: scalar + vector + bivector
scalar_part = 2.0 * jnp.eye(4, dtype=jnp.complex128)  # grade 0
vector_part = 1.5 * gammas[0] + 2.5 * gammas[1]       # grade 1
bivector_part = engine.wedge_product(gammas[2], gammas[3])  # grade 2

mixed = scalar_part + vector_part + bivector_part

# Project onto all grades
components = {
    grade: engine.project_grade(mixed, grade)
    for grade in range(5)
}

norms = {grade: float(jnp.linalg.norm(comp)) for grade, comp in components.items()}

# Pretty-print as a table
print(f"{'Grade':<6} {'Type':<15} {'Norm':<10}")
print("-" * 35)
grade_names = ["Scalar", "Vector", "Bivector", "Trivector", "Pseudoscalar"]
for grade, norm in norms.items():
    print(f"{grade:<6} {grade_names[grade]:<15} {norm:.4f}")

Grade  Type            Norm      
-----------------------------------
0      Scalar          4.0000
1      Vector          5.8310
2      Bivector        2.0000
3      Trivector       0.0000
4      Pseudoscalar    0.0000


**What this tells us:**

- **Grades 0, 1, 2** have non-zero norms because we explicitly added scalar, vector, and bivector parts to `mixed`.
- **Grades 3 and 4** are exactly zero (or machine epsilon) because our construction didn't include any trivector or pseudoscalar components.

This confirms that `project_grade` correctly isolates the geometric components of the multivector.

In [6]:
reconstructed = sum(components.values())
reconstruction_error = float(jnp.linalg.norm(mixed - reconstructed))
print("Reconstruction error ‖mixed − Σ grades‖:", f"{reconstruction_error:.2e}")

Reconstruction error ‖mixed − Σ grades‖: 0.00e+00


## 5. Cheat Sheet & Next Steps

### Cheat Sheet: Clifford Grades

- **Grade 0** – scalar  
- **Grade 1** – vector  
- **Grade 2** – bivector (2D oriented area)  
- **Grade 3** – trivector (3D volume)  
- **Grade 4** – pseudoscalar (4D "volume")  

### Next steps

- 👉 Go to **Notebook 3 – Basic QGT (Single Parameter)** to see how these structures feed into QGT.
- Or revisit the code here and:
  - Change the coefficients in `mixed`
  - Add a trivector / pseudoscalar term
  - See how the grade norms and reconstruction error change